# 2. Build a Recommendation System for Purchase Data

The scope of this notebook is 

- Code the Scoring Function
- Unit Test the Score
- Build the Dash Applimcation



## Settings

In [6]:
#Data Science
import pandas as pd


#Model Tracking
import mlflow
from mlflow.tracking import MlflowClient

from minio import Minio
from minio.error import ResponseError

from surprise import dump

import json
import pickle
import os

#Utils
from pprint import pprint
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

In [13]:
#Enviroment variables

outmodels = '../models/'
artefact_name = '4fa76aa5e00c413db3e23810a913dc8e'

## Download Model Artefact from the Registry

In [27]:
client = MlflowClient()
for regmodel in client.list_registered_models():
    regmodel_info = dict(regmodel)

# pprint(regmodel_info, indent=3)

champion=client.get_registered_model('Champion')
championid=champion.latest_versions[-1].run_id

art_list = [arts.path for arts in client.list_artifacts(championid, path=None)]

for art_path in art_list: 
    client.download_artifacts(championid, art_path, outmodels)

### Quick Check Model Content

In [33]:
modelpkl = "".join([outmodels, 'model/model_gm2xt4q.pkl'])
predictions, algo = dump.load(modelpkl)

print(predictions[0:10])
print('*'*100)
print(algo)

[Prediction(uid='10929', iid='261', r_ui=0.0, est=0.3747662865053879, details={'was_impossible': False}), Prediction(uid='2715', iid='256', r_ui=0.0, est=1.397448561679778, details={'was_impossible': False}), Prediction(uid='12786', iid='198', r_ui=0.0, est=0.34800521979059373, details={'was_impossible': False}), Prediction(uid='38', iid='119', r_ui=0.0, est=1.4081730858161146, details={'was_impossible': False}), Prediction(uid='8417', iid='2', r_ui=0.0, est=0.10200631019335304, details={'was_impossible': False}), Prediction(uid='5370', iid='293', r_ui=0.0, est=1.4426131167097673, details={'was_impossible': False}), Prediction(uid='225', iid='34', r_ui=0.0, est=0.6310815188817669, details={'was_impossible': False}), Prediction(uid='12930', iid='24', r_ui=2.0, est=0.5743177052276057, details={'was_impossible': False}), Prediction(uid='9481', iid='261', r_ui=0.0, est=0.45208051960508466, details={'was_impossible': False}), Prediction(uid='2410', iid='49', r_ui=0.0, est=0.7374420578118797

33397

### Single value

In [ ]:
test_sample = data_prep_4.head()
test_sample.head()

In [ ]:
uid = str(1007)  # raw user id (as in the ratings file). They are **strings**!
iid = str(0)  # raw item id (as in the ratings file). They are **strings**!

# get a prediction for specific users and items.
pred = algo_best.predict(uid, iid, r_ui=13, verbose=True)

### Score the entire dataset

In [ ]:
data_prep_5['predictions'] = data_prep_5.apply(lambda row:algo_best.predict(row['userID'], 
                     row['itemID'], row['rating']), axis = 1)
data_prep_5.head()
# predictions=list(data_prep_5['predictions'])

### Top 10 Recommended Items


In [ ]:
def get_top_n(predictions, n=10):
    
    '''Return the top-N recommendation for each user from a set of predictions.
    Args:
        predictions(list of Prediction objects): The list of predictions, as
            returned by the test method of an algorithm.
        n(int): The number of recommendation to output for each user. Default
            is 10.
    Returns:
    A dict where keys are user (raw) ids and values are lists of tuples:
        [(raw item id, rating estimation), ...] of size n.
    '''

    # First map the predictions to each user.
    top_n = defaultdict(list)
    for uid, iid, true_r, est, _ in predictions:
        top_n[uid].append((iid, est))

#     # Then sort the predictions for each user and retrieve the k highest ones.
    for uid, user_ratings in top_n.items():
        user_ratings.sort(key=lambda x: x[1], reverse=True)
        top_n[uid] = user_ratings[:n]

    return top_n

def get_top_n_ui(top, ui):
    try:
        return {k:v for k,v in top.items() if ui==k}
    except ValueError: # user was not part of the trainset
        return 0

In [ ]:
print(get_top_n_ui(get_top_n(predictions, n=10), '20400'))